In [9]:
import scanpy as sc
import numpy as np
import scipy as sp
from scipy import io
import pandas as pd
import pickle as pkl
import itertools
import os
import sys
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import colors
import seaborn as sns
from gprofiler import gprofiler
# from gprofiler import GProfiler

import rpy2.rinterface_lib.callbacks
import logging

from rpy2.robjects import pandas2ri
import anndata2ri
from dask import dataframe as dd
import dask   

sc.settings.verbosity = 3

In [10]:
sample_strings = ['control', '12h_afterCrush', '1d_afterCrush', '2d_afterCrush', '4d_afterCrush', '1w_afterCrush', '2w_afterCrush']
file_base = '/home/ionelia/pycharm-projects/master/semester2/internshipISTaustria/TranscriptomicsOptimalControl/Tran_RGC_scRNA/unzipped/GSE137398_ONCRGCs_'
data_file_end = '_count_mat.csv.gz'
data_file_end = '_count_mat.csv'

In [11]:
%%timeit

control, crushed12, crushed1d = None, None, None
for i in np.arange(len(sample_strings)-6):
    sample = sample_strings[i]
    data_file = file_base+sample+data_file_end
    print(i, data_file)
    
    adata_tmp = sc.read_csv(data_file, first_column_names=True)
    adata_tmp = adata_tmp.transpose()
    
    barcodes = ['%s_%d'%(sample, xx) for xx in np.arange(len(adata_tmp.obs))]
    adata_tmp.obs.index = barcodes
    adata_tmp.obs['sample'] = [sample]*adata_tmp.n_obs
    print(adata_tmp.obs.keys())
    
    if i!=0:
        # Concatenate to main adata object
        adata = adata.concatenate(adata_tmp)
        if i==1:
            crushed12 = adata_tmp
        else:   
            crushed1d = adata_tmp
    else:
        adata = adata_tmp.copy()
        contorl = adata_tmp
        
    del(adata_tmp)
    break

0 /home/ionelia/pycharm-projects/master/semester2/internshipISTaustria/TranscriptomicsOptimalControl/Tran_RGC_scRNA/unzipped/GSE137398_ONCRGCs_control_count_mat.csv
Index(['sample'], dtype='object')
0 /home/ionelia/pycharm-projects/master/semester2/internshipISTaustria/TranscriptomicsOptimalControl/Tran_RGC_scRNA/unzipped/GSE137398_ONCRGCs_control_count_mat.csv
Index(['sample'], dtype='object')
0 /home/ionelia/pycharm-projects/master/semester2/internshipISTaustria/TranscriptomicsOptimalControl/Tran_RGC_scRNA/unzipped/GSE137398_ONCRGCs_control_count_mat.csv
Index(['sample'], dtype='object')
0 /home/ionelia/pycharm-projects/master/semester2/internshipISTaustria/TranscriptomicsOptimalControl/Tran_RGC_scRNA/unzipped/GSE137398_ONCRGCs_control_count_mat.csv
Index(['sample'], dtype='object')
0 /home/ionelia/pycharm-projects/master/semester2/internshipISTaustria/TranscriptomicsOptimalControl/Tran_RGC_scRNA/unzipped/GSE137398_ONCRGCs_control_count_mat.csv
Index(['sample'], dtype='object')
0 /ho

In [4]:
%%timeit
sample = sample_strings[0]
data_file = file_base+sample+data_file_end

adata_tmp_ori = sc.read_csv(data_file, first_column_names=True)
adata_tmp = adata_tmp_ori.transpose()

barcodes = ['%s_%d'%(sample, xx) for xx in np.arange(len(adata_tmp.obs))]
adata_tmp.obs.index = barcodes
adata_tmp.obs['sample'] = [sample]*adata_tmp.n_obs
print(adata_tmp.obs.keys())

Index(['sample'], dtype='object')
Index(['sample'], dtype='object')
Index(['sample'], dtype='object')
Index(['sample'], dtype='object')
Index(['sample'], dtype='object')
Index(['sample'], dtype='object')
Index(['sample'], dtype='object')
Index(['sample'], dtype='object')
55.3 s ± 1.03 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
sample

NameError: name 'sample' is not defined

In [4]:
contorl

AnnData object with n_obs × n_vars = 17887 × 40790
    obs: 'sample'

In [7]:
dask_df_control = dd.read_csv(file_base+sample_strings[0]+data_file_end,  sample=1000000)
dask_df_control.head()

,Unnamed: 0,CtC57CD45CD90P1_AAACCTGAGCTAACAA-1,CtC57CD45CD90P1_AAACCTGCACGTAAGG-1,CtC57CD45CD90P1_AAACCTGCAGACAAAT-1,CtC57CD45CD90P1_AAACCTGTCCGCGGTA-1,CtC57CD45CD90P1_AAACGGGAGCTCTCGG-1,CtC57CD45CD90P1_AAACGGGCAAGCGATG-1,CtC57CD45CD90P1_AAACGGGTCGGATGGA-1,CtC57CD45CD90P1_AAAGCAACAACTGGCC-1,CtC57CD45CD90P1_AAAGCAACAATCCGAT-1,...,CtVgCD90R1_TTTGTCAAGTCAAGGC-1,CtVgCD90R1_TTTGTCACATACGCTA-1,CtVgCD90R1_TTTGTCAGTAGCGCAA-1,CtVgCD90R1_TTTGTCAGTTCGGCAC-1,CtVgCD90R1_TTTGTCAGTTTCCACC-1,CtVgCD90R1_TTTGTCATCAAGGTAA-1,CtVgCD90R1_TTTGTCATCCCATTAT-1,CtVgCD90R1_TTTGTCATCCCTAACC-1,CtVgCD90R1_TTTGTCATCCTTAATC-1,CtVgCD90R1_TTTGTCATCGCTAGCG-1
0,4933401J01Rik,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Gm26206,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Gm1992,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Gm10568,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Gm38385,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
dask_csv = dd.to_csv(dask_df_control) 

TypeError: to_csv() missing 1 required positional argument: 'filename'

In [ ]:
dask_df_control